# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [1]:
!nvidia-smi

Thu Jul 20 17:49:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --upgrade gdown

# Main link
!gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Bqckup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

!unzip -q libriphone.zip
!ls libriphone

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc
From (redirected): https://drive.google.com/uc?id=1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc&confirm=t&uuid=b89d1088-3aa7-4723-86f7-1f1d7e854780
To: /content/libriphone.zip
100% 479M/479M [00:07<00:00, 67.4MB/s]
feat  test_split.txt  train_labels.txt	train_split.txt


# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [2]:
# Numerical Operations
import random
import numpy as np

# Reading/Writing Data
import os
import pandas as pd

# Garbage Collection
import gc

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Ref: https://github.com/Bond-SYSU/pytorch-summary
from torchsummary import summary

# Visualize the training information
import matplotlib.pyplot as plt

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    """ Get n frames' features. """
    assert concat_n % 2 == 1                # Total n frames must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size()         # Dimension: (seq_len, feature_dim)
    x = x.repeat(1, concat_n)               # Repeats tensor x along the specified dimensions.
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # Dimension: (concat_n, seq_len, feature_dim)
    mid = (concat_n // 2)                   # Get the middle of n frames(We want to predict).
    for r_idx in range(1, mid+1):
        x[mid + r_idx] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2)               # Dimension: (seq_len, concat_n, feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41                          # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'valid') else 'test'

    # Get train / valid label dictionary
    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'valid':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)                         # Fix the random seed.
        random.shuffle(usage_list)                          # Training data shuffled.
        percent = int(len(usage_list) * train_ratio)        # Border index between training and validation data.
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:] # Get training / validation data
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]  # Strip newline in dataset.
    print(f"[Dataset] - # phone classes: {str(class_num)}, number of utterances for {split}: {str(len(usage_list))}")

    max_len = 3000000
    X = torch.empty(max_len, concat_nframes, 39)            # Create an empty tensor.
    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)          # Create an empty tensor.

    idx = 0
    for fname in tqdm(usage_list):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))       # Load features.
        cur_len = len(feat)                                                 # Length of features
        feat = concat_feat(feat, concat_nframes)                            # Get features after concatenating n frames.
        # Get train / valid labels.
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat                                     # Copy features to X.
        if mode != 'test':
            y[idx: idx + cur_len] = label                                   # Copy label to y.

        idx += cur_len

    X = X[:idx, :]      # Discard other empty tensor.
    if mode != 'test':
        y = y[:idx]     # Discard other empty tensor.

    print(f"[INFO] {split} set")
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X, y
    else:
        return X

# Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [5]:
import torch.nn as nn

class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
        dropout = 0.5,
        batch_first = True,
        bidirectional = True,
    )
    #self.batchnorm = nn.BatchNorm1d(hidden_size*2)
    self.act = nn.ReLU()
    self.dropout = nn.Dropout(0.5)
    self.fc = nn.Linear(hidden_size*2, num_classes)

  def forward(self, x):
    out, _ = self.lstm(x)
    #out = self.batchnorm(out)
    out = self.act(out)
    out = self.dropout(out)
    out = out[:, -1, :]
    out = self.fc(out)
    return out


# Hyper-parameters

In [6]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 23   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 5785          # random seed
batch_size = 256        # batch size
num_epoch = 10         # the number of training epoch
learning_rate = 1e-3      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39  # the input dim of the model, you should not change the value
hidden_layers = 4          # the number of hidden layers
hidden_dim = 128       # the hidden dim

# Dataloader

In [7]:
# Preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
valid_X, valid_y = preprocess_data(split='valid', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# Get dataset
train_set = LibriDataset(train_X, train_y)
valid_set = LibriDataset(valid_X, valid_y)

# Remove raw feature to save memory
del train_X, train_y, valid_X, valid_y
gc.collect()

# Get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=2, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 3428


100%|██████████| 3428/3428 [00:18<00:00, 185.22it/s]


[INFO] train set
torch.Size([2116368, 23, 39])
torch.Size([2116368])
[Dataset] - # phone classes: 41, number of utterances for valid: 858


100%|██████████| 858/858 [00:03<00:00, 260.58it/s]


[INFO] valid set
torch.Size([527790, 23, 39])
torch.Size([527790])


# Training

In [8]:
# create model, define a loss function, and optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=hidden_layers, num_classes=41).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(valid_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)

            loss = criterion(outputs, labels)

            _, val_pred = torch.max(outputs, 1)
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(valid_set):3.5f} loss: {val_loss/len(valid_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(valid_set):.5f}')


100%|██████████| 2062/2062 [00:31<00:00, 64.56it/s]


[001/010] Train Acc: 0.63862 Loss: 1.22886 | Val Acc: 0.72597 loss: 0.89743
saving model with acc 0.72597


100%|██████████| 2062/2062 [00:31<00:00, 64.84it/s]


[002/010] Train Acc: 0.73974 Loss: 0.86535 | Val Acc: 0.75166 loss: 0.82461
saving model with acc 0.75166


100%|██████████| 2062/2062 [00:31<00:00, 64.52it/s]


...

[008/010] Train Acc: 0.82037 Loss: 0.57928 | Val Acc: 0.76834 loss: 0.86244
saving model with acc 0.76834


100%|██████████| 2062/2062 [00:31<00:00, 65.49it/s]


[009/010] Train Acc: 0.82604 Loss: 0.56092 | Val Acc: 0.76724 loss: 0.87189


100%|██████████| 2062/2062 [00:31<00:00, 64.86it/s]

[010/010] Train Acc: 0.83051 Loss: 0.54484 | Val Acc: 0.76908 loss: 0.87719
saving model with acc 0.76908


In [9]:
del train_set, valid_set
del train_loader, valid_loader
gc.collect()

0

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [10]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


100%|██████████| 1078/1078 [00:05<00:00, 186.34it/s]

[INFO] test set
torch.Size([646268, 23, 39])


In [11]:
# load model
model = LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=hidden_layers, num_classes=41).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [12]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 2525/2525 [00:35<00:00, 70.15it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [13]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))